In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import datetime

In [3]:
## 한글 폰트
import matplotlib.font_manager as fm
import matplotlib as mpl

font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')

for font in font_list:
    if 'Nanum' in font:
        print(font)
        
print()
print(mpl.matplotlib_fname())

plt.rcParams["font.family"] = 'NanumGothic'

## 마이너스 부호
mpl.rcParams['axes.unicode_minus'] = False

/Library/Fonts/NanumSquareRegular.ttf
/Library/Fonts/NanumMyeongjoExtraBold.ttf
/Library/Fonts/NanumSquareExtraBold.ttf
/Library/Fonts/NanumGothicLight.ttf
/Library/Fonts/NanumBarunpenBold.ttf
/Library/Fonts/NanumSquareRoundB.ttf
/Library/Fonts/NanumGothicExtraBold.ttf
/Library/Fonts/NanumBarunGothicBold.ttf
/System/Library/AssetsV2/com_apple_MobileAsset_Font6/ee9e9584cd1a5f676e03a9efcc80783bc7e12e2c.asset/AssetData/NanumGothic.ttc
/Library/Fonts/NanumSquareRoundR.ttf
/Library/Fonts/NanumSquareLight.ttf
/Library/Fonts/NanumGothic.ttf
/Library/Fonts/NanumBarunGothicUltraLight.ttf
/Library/Fonts/NanumBarunGothicLight.ttf
/Library/Fonts/NanumPen.ttf
/Library/Fonts/NanumBrush.ttf
/Library/Fonts/NanumMyeongjo.ttf
/Library/Fonts/NanumSquareBold.ttf
/Library/Fonts/NanumBarunGothic.ttf
/Library/Fonts/NanumMyeongjoBold.ttf
/Library/Fonts/NanumSquareRoundL.ttf
/Library/Fonts/NanumSquareRoundEB.ttf
/Library/Fonts/NanumBarunpenRegular.ttf
/System/Library/AssetsV2/com_apple_MobileAsset_Font6/3b0bb0

In [4]:
## riot API
kr_url = 'https://kr.api.riotgames.com/'
asia_url = 'https://asia.api.riotgames.com/'
api_key = 'RGAPI-b5e53fa7-879d-4cd6-8dd0-ccb411e253a4'
api = '?api_key='+api_key

# 1. 티어별 소환사명 목록 만들기

## Challenger to Master

In [9]:
ch_queue_req = requests.get(f'{kr_url}lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5{api}')
gm_queue_req = requests.get(f'{kr_url}lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5{api}')
ms_queue_req = requests.get(f'{kr_url}lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5{api}')

In [14]:
challengers = pd.json_normalize(ch_queue_req.json()['entries'])
grandmasters = pd.json_normalize(gm_queue_req.json()['entries'])
masters = pd.json_normalize(ms_queue_req.json()['entries'])

ch_to_ms = pd.concat([challengers, grandmasters, masters], ignore_index=True)

print('# of summoners:    ', len(ch_to_ms))
print('# of challengers:  ', len(challengers))
print('# of grandmasters: ', len(grandmasters))
print('# of masters:      ', len(masters))
print()

ch_to_ms.head()

# of summoners:     4251
# of challengers:   300
# of grandmasters:  700
# of masters:       3251



,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,2OMUVkdEFjCQ-X9cnKnVzutigIzx39Mo62QlK5qcFIW2E5g,T1 Zeus,1143,I,459,397,True,False,False,False
1,_iG58wg8kGfppZxQQITHQOvt8IvFVZzfSvFX88TrCcMrjp...,tan hua yi xian,977,I,877,807,False,False,False,False
2,xy686tdArwnwtqijpMWjZAYtzM45g3xTZJ1ncI4PeaD18Os,BRO Loopy,805,I,757,705,False,False,True,False
3,Jg00lT8Ynrkiop9yUW-yWgB0j95COw7wao6_w5pJnuJusOs,Gen G Bdd,1011,I,224,171,True,False,False,False
4,2F6koT1BM5kSxA6y7E_VL--jS3XZ-3O8vCNoHayVUimOjB...,wu guan feng yue,1320,I,727,647,True,False,False,False


In [15]:
ch_to_ms.to_csv('./data/ch_to_ms.csv')

# 2. 소환사명으로 소환사 데이터 불러오기

In [21]:
summoners_df = pd.DataFrame()

i = 0
N = len(ch_to_ms.summonerName.values)

In [22]:
while(i < N):
    
    sn = ch_to_ms.summonerName.values[i]
    
    try:
        summoner_req = requests.get(f'{kr_url}lol/summoner/v4/summoners/by-name/{sn}{api}')
        summoner_df = pd.json_normalize(summoner_req.json())
        if i % 100 == 0:
            print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, summoner_df.name[0]))
    except Exception as e:
        if summoner_df['status.status_code'][0] == 404:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            pass
        elif summoner_df['status.status_code'][0] == 429:
            time.sleep(120)
            continue
        else:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            time.sleep(5)
            continue
        
    summoners_df = summoners_df.append(summoner_df, ignore_index=True)
    i += 1

05:47:11       0: T1 Zeus
05:47:41     100: BJ갈배
05:50:10     200: 돔 심
05:52:46     300: 종갈통
05:55:18     400: IreliaChallenger
05:57:48     500: Golden mole
06:00:18     600: 백지헌백지헌백지
06:02:51     700: tzvcxy
06:05:21     800: 비밀프로젝트
06:07:52     900: 처음부터다시하다
06:10:22    1000: 주나박
06:12:52    1100: 카시는작고소중해
06:15:22    1200: Shadow Ming9
06:17:53    1300: 수면소년
06:20:23    1400: ok동
06:22:54    1500: 귀신 녹턴
06:25:25    1600: 뱀정운
06:27:55    1700: 05ADC
06:30:27    1800: Odelia flower
06:32:57    1900: WinDei
06:35:28    2000: 핏 라
06:38:00    2100: zhongdan Irelia
06:40:31    2200: zxczxczv
06:43:02    2300: 허니바니
06:45:33    2400: 은결쌤
06:48:03    2500: pigwsy
06:50:34    2600: hylt hylt
06:53:04    2700: duinixindong
06:55:34    2800: 정점의 피지컬
06:58:05    2900: 둥가락
07:00:36    3000: Venta
07:03:07    3100: zdhl
07:05:38    3200: 원딜깡패오민혁
07:08:09    3300: 쁘 뽀
07:10:41    3400: 야호붕붕대장
07:13:11    3500: 망토가 펄럭
07:15:42    3600: wushugexingcheng
07:18:14    3700: MADE IN JUNGLE
07:20:45    3

In [23]:
summoners_df.to_csv('./data/summoners_df.csv')
summoners_df.head()

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,status.message,status.status_code
0,2OMUVkdEFjCQ-X9cnKnVzutigIzx39Mo62QlK5qcFIW2E5g,d8iLopH4-JkPo4S47OtjC_VqjI3QRR1KvNs4UqmVO5V9kCA,ODWTUNHDMtK9C0h1_ObdjJ6ZDkij4DJNAAtWMrwiIQ-TyS...,T1 Zeus,5040.0,1.629800e+12,367.0,NaN,NaN
1,_iG58wg8kGfppZxQQITHQOvt8IvFVZzfSvFX88TrCcMrjp...,3KmoGjA5MmatwKUPvhZVlYXtmIxx4TIs7CaLCUs3RwbF4k...,hcF9febiHTR-26ft4VinC0wIAIoHr2lpP0ClGOxOCZ5zAl...,tan hua yi xian,7.0,1.630063e+12,152.0,NaN,NaN
2,xy686tdArwnwtqijpMWjZAYtzM45g3xTZJ1ncI4PeaD18Os,vkmZnwY1b50o7XtwItxtDit2T4HYQ36aeFPY5RRwY28Vyg...,D0RYGxX15DItJnF8nsu4ushlBuFRLLRzUZypMtY9pV8JdH...,BRO Loopy,4802.0,1.630074e+12,298.0,NaN,NaN
3,Jg00lT8Ynrkiop9yUW-yWgB0j95COw7wao6_w5pJnuJusOs,5PYA6Cqr8LQOX6F-Mt63cU-mlab1d0vJPO6euyCaLPqMAyk,ftmmwwL7XStkrgAlIk7QAotgJseK01_B2RNZEHEyOt0sbA...,Gen G Bdd,7.0,1.630063e+12,208.0,NaN,NaN
4,2F6koT1BM5kSxA6y7E_VL--jS3XZ-3O8vCNoHayVUimOjB...,Hdoz8u8Oo6V7R7K6ZnowMdiXyp7OX25QWLiaIrdbOCWJ45...,jC2GVl9V2hssOLVNnWAjrqGVIbt8mlW3GyPHW68FH2Clvv...,wu guan feng yue,5034.0,1.630056e+12,204.0,NaN,NaN


In [24]:
# 합치기
summoners_df.drop(columns=['id'], inplace=True)
summoners_df.rename(columns={'name':'summonerName'}, inplace=True)

SUMMONERS = pd.merge(ch_to_ms, summoners_df, on='summonerName', how='inner')
SUMMONERS.drop(columns=['status.message', 'status.status_code'], inplace=True)
SUMMONERS.to_csv('./data/ch_to_ms.csv')
SUMMONERS.tail()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId,puuid,profileIconId,revisionDate,summonerLevel
4177,22dvCHs0D5mPfP_YHwO0HNXOv2GecybQJocqAMXYEU9rZ1o,클레브,103,I,162,141,False,False,False,False,ZVG7KbbUOpCyXA1afMIDleHnwULxVgtpvZs8TqUwhlepds8,yuaXZctRxAVOmiSxo6-bt-GNjrXs1UfS668oGr2J0jAHxe...,4535.0,1.629970e+12,162.0
4178,6jbvZfUaLfHtrTh3JbDM4-SnnNc1usfeUiQVmkUd35Wyks4,동규킹,0,I,199,127,False,False,True,False,VteggI1LcI4KGKBa98CUXxU77NmAmMHFd2i1trTdfXln-eI,PDo_SsDEfMKGtqmgjRMSNRbysnNuswG7jeE9ScCOoLPkyW...,29.0,1.630097e+12,78.0
4179,VMKJvEVH3Ift2UCua5kaB0o6HNgucXOx70WASybSH16lybc,과묵한 왕자,97,I,296,281,True,False,False,True,VjApUuwfy50XeIjV_hEZqzPkORalOB1F2frhkOy-ibuYv5I,01jdQ5AQDnyfhg0Nk9kAJRcyeAyjtuINtQbYy3ncLH0rkD...,4022.0,1.629791e+12,394.0
4180,1lht-9Kq5AgxGAWdpnKiWqVsK03wNlH2Fa9b_-pjamQc1w...,너네랑 어케이겨,220,I,100,75,True,False,False,False,hha365pN9mUvblXAyDGDm6WCjrPLKACKRaC0u77d_naQ6h...,ECKDO-8BR7GlXmoASiYvE5JtUVMyeTNMDPDbZHLMDAeVF0...,5023.0,1.629809e+12,47.0
4181,VzBqUa3lVqp2GYhz18NitbcDX1hZ-symSBATjmww3pCgsiIM,rocky2,297,I,234,202,False,False,False,True,ponKVzyxadKemOoWVnxME-1ieoJMy6SZjad0UCwqinZkq5...,tYYmnFekAu0XllV-P7MYbUxJcf_cM3HWvgCyBfSyLLYHHg...,4302.0,1.629979e+12,73.0


# 3. 소환사 계정 ID로 match 데이터 불러오기

In [25]:
SUMMONERS = pd.read_csv('./data/ch_to_ms.csv')

In [26]:
matches_set = set()

N = len(SUMMONERS)
i = 0

In [27]:
while(i < N):
    
    ai = SUMMONERS.accountId.values[i]
    
    try:
        matches_req = requests.get(f'{kr_url}lol/match/v4/matchlists/by-account/{ai}{api}')
        matches_df = pd.json_normalize(matches_req.json()['matches'])
        
        matches_set.update(matches_df.gameId.values)
        if i % 100 == 0:
            print('{0}   {1:5d}: {2:5d}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, len(matches_set)))
        
        i += 1
    
    except Exception as e:
        if matches_req.status_code == 429:
            time.sleep(120)
            continue
        else:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            time.sleep(5)
            i += 1
            continue

07:31:38       0:   100
07:34:27   'matches' 
07:34:32     100:  6888
07:34:42   'matches' 
07:34:52   'matches' 
07:37:29     200: 10917
07:40:16     300: 15507
07:40:38   'matches' 
07:43:10     400: 21524
07:45:59     500: 26178
07:48:49     600: 30175
07:48:53   'matches' 
07:51:43     700: 33660
07:54:33     800: 37306
07:54:46   'matches' 
07:57:20   'matches' 
07:57:32     900: 40318
08:00:22    1000: 45311
08:03:05   'matches' 
08:03:14    1100: 53027
08:06:03    1200: 60414
08:08:51    1300: 67745
08:09:03   'matches' 
08:11:43    1400: 74600
08:14:33    1500: 80656
08:17:21    1600: 87012
08:20:10    1700: 93016
08:22:41   'matches' 
08:23:03    1800: 98836
08:25:53    1900: 104025
08:28:47    2000: 109602
08:28:54   'matches' 
08:31:38   'matches' 
08:31:45    2100: 114862
08:34:35    2200: 120439
08:34:51   'matches' 
08:37:30    2300: 125292
08:40:06   'matches' 
08:40:25    2400: 130013
08:43:14    2500: 134208
08:43:25   'matches' 
08:46:07    2600: 138674
08:48:55    27

In [28]:
with open('./data/matches_set.txt', 'w') as file:
    for match in list(matches_set):
        file.write(str(match)+'\n')

# 4. match id로 match 데이터 불러오기

In [5]:
matches_df = pd.read_csv('./data/matches_set.txt', header=None, names=['match_id'])
matches_list = matches_df['match_id'].values
print(len(matches_list))

203213


In [9]:
MATCHES = pd.DataFrame()

N = len(matches_list)
j = 0

In [11]:
while(j < N):
    
    mi = str(matches_list[j])
    
    try:
        match_req = requests.get(f'{kr_url}lol/match/v4/matches/{mi}{api}')
        match_df = pd.json_normalize(match_req.json())
        gameId = match_df.gameId[0]
#         print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, match_df.gameId[0]))
    except Exception as e:
        if match_req.status_code == 405 or match_req.status_code == 403:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), match_req.status_code))
            pass
        elif match_req.status_code == 429:
            time.sleep(120)
            continue
        else:
            print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
            time.sleep(5)
            continue
    
    try:
        #=============================================================
        #match_df 정리
        #=============================================================
        #match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
        #                   'mapId', 'queueId'], inplace=True)

        prtcp_key_list = ['championId', 'spell1Id', 'spell2Id']
        prtcpId_key_list = ['accountId', 'summonerName', 'summonerId']


        #승리 팀--------------------------------------
        team1 = match_df['teams'].values[0][0]['win']
        team2 = match_df['teams'].values[0][1]['win']

        if team1 == 'Win':
            match_df['win'] = 'team1'
        else:
            match_df['win'] = 'team2'
        #---------------------------------------------


        #lane 결정-------------------------------------------
        def select_lane(lane, role):
            if lane == 'TOP':
                return 'TOP'
            elif lane == 'JUNGLE':
                return 'JUG'
            elif lane == 'MIDDLE':
                return 'MID'
            elif lane == 'BOTTOM' and role == 'DUO_CARRY':
                return 'ADC'
            elif lane == 'BOTTOM' and role == 'DUO_SUPPORT':
                return 'SUP'
            else:
                return None
        #---------------------------------------------------


        #team 1 데이터 정리----------------------------------------------------
        for i in range(5):
            lane = match_df['participants'].values[0][i]['timeline']['lane']
            role = match_df['participants'].values[0][i]['timeline']['role']

            lane = select_lane(lane, role)
            if lane == None:
                lane = str(i)

            for key in prtcp_key_list:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participants'].values[0][i][key]

            for key in prtcpId_key_list:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participantIdentities'].values[0][i]['player'][key]

            for key in ['lane', 'role']:
                match_df['team1_{0}_{1}'.format(i+1, key)] = match_df['participants'].values[0][i]['timeline'][key]
                
            match_df['team1_{0}_position'.format(i+1)] = lane
        #----------------------------------------------------------------------

        #team 2 데이터 정리-----------------------------------------------------
        for i in range(5,10):
            lane = match_df['participants'].values[0][i]['timeline']['lane']
            role = match_df['participants'].values[0][i]['timeline']['role']

            lane = select_lane(lane, role)

            for key in prtcp_key_list:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participants'].values[0][i][key]

            for key in prtcpId_key_list:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participantIdentities'].values[0][i]['player'][key]

            for key in ['lane', 'role']:
                match_df['team2_{0}_{1}'.format(i-4, key)] = match_df['participants'].values[0][i]['timeline'][key]
                    
            match_df['team2_{0}_position'.format(i-4)] = lane
        #-----------------------------------------------------------------------
        
        match_df.drop(columns=['teams', 'participants', 'participantIdentities'], inplace=True)
        MATCHES = MATCHES.append(match_df, ignore_index=True)
        
    except Exception as e:
        print('{0}   {1:5d}: {2} {3}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, match_df.gameId[0], e))
        
    j += 1
    
    if (j%100) == 0:
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, gameId))
        MATCHES.to_csv('./data/matches.csv')

20:53:51    6000: 5327830511
20:56:43    6100: 5413813982
20:59:35    6200: 5359812596
21:02:31    6300: 5341462829
21:02:55    6332: 5395988862 'summonerId'
21:05:37    6400: 5248664091
21:05:43   'DataFrame' object has no attribute 'gameId' 
21:05:52   'DataFrame' object has no attribute 'gameId' 
21:08:34    6463: 5415912106 'summonerId'
21:08:49   'DataFrame' object has no attribute 'gameId' 
21:09:15   'DataFrame' object has no attribute 'gameId' 
21:09:49    6500: 5337268985
21:10:38   'DataFrame' object has no attribute 'gameId' 
21:10:53   'DataFrame' object has no attribute 'gameId' 
21:11:10   'DataFrame' object has no attribute 'gameId' 
21:12:52   HTTPSConnectionPool(host='kr.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v4/matches/5374493678?api_key=RGAPI-b5e53fa7-879d-4cd6-8dd0-ccb411e253a4 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8bc86598b0>: Failed to establish a new connection: [Errno 60] Operation time

AttributeError: 'DataFrame' object has no attribute 'gameId'

In [111]:
print(j)

39171


In [109]:
MATCHES.tail()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,platformId,queueId,seasonId,...,team2_4_lane,team2_4_role,team2_5_championId,team2_5_spell1Id,team2_5_spell2Id,team2_5_accountId,team2_5_summonerName,team2_5_summonerId,team2_5_lane,team2_5_role
38966,1593509795770,2137,4479861189,CLASSIC,MATCHED_GAME,10.13.326.4870,11,KR,420,13,...,BOTTOM,DUO_SUPPORT,245,4,11,AqraIK5579cEijNovrM1qSIRAnhfXi8idismhRMSVwftEq...,dimemaythgbedha,FIZG8K2jYKJXpWws-etWKeqE6MQ7byQ7ZHOwK35YC8c1xQU,JUNGLE,NONE
38967,1594374792223,1082,4499784140,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,420,13,...,NONE,DUO_SUPPORT,106,11,4,iUciSBA_ccas_6QrCTg9MOT3qPUHfzAdpfwkYVmOcJvQwA...,Keep calm and C,mGc_NpEOPTEtfJtzWx0oot7yBigONZfX4mBMmvPAEJyv4mc,NONE,DUO_SUPPORT
38968,1590937805122,2023,4420092364,CLASSIC,MATCHED_GAME,10.11.322.2991,11,KR,430,13,...,BOTTOM,DUO_CARRY,39,14,4,IuougcPWMMp4kWlzIWAdJOgmSLrwAYdjTJqMmS3b4mE8je...,라넌큘러스,dygPkJ-IRrUGHSzQoNXqUzoxyBrQ-FJWqF2UKj8E5uCSGic,MIDDLE,SOLO
38969,1592999354990,1754,4467802580,CLASSIC,MATCHED_GAME,10.13.325.7485,11,KR,420,13,...,BOTTOM,DUO_SUPPORT,266,4,12,gF8YqddwaqXsn_zkzzjD1MN3bIRPlp4uZVvmDpUGdXB9NZM,운도리,88maOsuXTEqdGgTmF-9NrwLM35LWfWaXiKN73mItbKntAQ0,TOP,SOLO
38970,1595745787447,1181,4532290004,NEXUSBLITZ,MATCHED_GAME,10.15.328.8759,21,KR,1300,13,...,NONE,DUO_SUPPORT,67,4,21,30JPMzxxQTMg993W0QrCKrTtiPXB33ztHUHd7plu8QXa,꿈꾸는 도토리,8Wt61XuiQ_df-XuOVNQVy2tM_dlj8fKkwzU9rDgQqs2zoQ,NONE,DUO


# 매치 데이터 정리

* gameId
* seasonId
* gameVersion
* participants
    - championId
    - spell1Id
    - spell2Id
* participantIdentities
    - accountId
    - summonerName
    - summonerId

In [110]:
match_df = json_normalize(match_req.json())
match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
                       'mapId', 'queueId'], inplace=True)
# match_df.set_index('gameId', inplace=True)
match_df.head()

,gameDuration,gameId,gameVersion,participantIdentities,participants,platformId,seasonId,teams
0,1867,4530461969,10.15.328.8759,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."


In [119]:
match_df = json_normalize(match_req.json())
match_df.drop(columns=['gameCreation', 'gameMode', 'gameType',
                       'mapId', 'queueId'], inplace=True)

prtcp_key_list = ['championId', 'spell1Id', 'spell2Id']
prtcpId_key_list = ['accountId', 'summonerName', 'summonerId']

team1 = match_df['teams'].values[0][0]['win']
team2 = match_df['teams'].values[0][1]['win']

if team1 == 'Win':
    match_df['win'] = 'team1'
else:
    match_df['win'] = 'team2'
    
def select_lane(lane, role):
    if lane == 'TOP':
        return 'TOP'
    elif lane == 'JUNGLE':
        return 'JUG'
    elif lane == 'MIDDLE':
        return 'MID'
    elif lane == 'BOTTOM' and role == 'DUO_CARRY':
        return 'ADC'
    elif lane == 'BOTTOM' and role == 'DUO_SUPPORT':
        return 'SUP'
    else:
        return None
    
for i in range(5):
    lane = match_df['participants'].values[0][i]['timeline']['lane']
    role = match_df['participants'].values[0][i]['timeline']['role']
    
    lane = select_lane(lane, role)
    
    for key in prtcp_key_list:
        match_df['team1_{0}_{1}'.format(lane, key)] = match_df['participants'].values[0][i][key]
        
    for key in prtcpId_key_list:
        match_df['team1_{0}_{1}'.format(lane, key)] = match_df['participantIdentities'].values[0][i]['player'][key]
        
for i in range(5,10):
    lane = match_df['participants'].values[0][i]['timeline']['lane']
    role = match_df['participants'].values[0][i]['timeline']['role']
    
    lane = select_lane(lane, role)
    
    for key in prtcp_key_list:
        match_df['team2_{0}_{1}'.format(lane, key)] = match_df['participants'].values[0][i][key]
        
    for key in prtcpId_key_list:
        match_df['team2_{0}_{1}'.format(lane, key)] = match_df['participantIdentities'].values[0][i]['player'][key]
        
    
match_df.drop(columns=['teams', 'participants', 'participantIdentities'], inplace=True)
match_df.set_index('gameId', inplace=True)
match_df.head()

,gameDuration,gameVersion,platformId,seasonId,win,team1_TOP_championId,team1_TOP_spell1Id,team1_TOP_spell2Id,team1_TOP_accountId,team1_TOP_summonerName,...,team2_ADC_spell2Id,team2_ADC_accountId,team2_ADC_summonerName,team2_ADC_summonerId,team2_SUP_championId,team2_SUP_spell1Id,team2_SUP_spell2Id,team2_SUP_accountId,team2_SUP_summonerName,team2_SUP_summonerId
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,1867,10.15.328.8759,KR,13,team1,68,12,4,B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA,사 용 수,...,7,1obHO2Y4DAZlobK7396eUH3-L85RjjkWHANvsohiOaVti2...,wohennangu0,zIHvAi4E8ZKpdMKsyY4nqFZ6tO4KesA7go4W6Nu2FeClxwQ,875,4,14,04bIyaoAJLviZ-6-Bofb7zYVWeU6Tnk5uAz-43CMLdE94G...,실망자,cdya4bpOdk15ISCADNsLWs0GolBDkHJ5-zgxU-wsJz0zCY0


In [130]:
match_train_df = match_df.drop(columns=['win', 'gameDuration', 'gameVersion', 'platformId', 'seasonId'])

orders = []

for team in ['team1', 'team2']:
    for lane in ['TOP', 'JUG', 'MID', 'ADC', 'SUP']:
        match_train_df.drop(columns=[team+'_'+lane+'_accountId', 
                                     team+'_'+lane+'_summonerName', 
                                     team+'_'+lane+'_summonerId'], 
                            inplace=True)
        
        orders.append(team+'_'+lane+'_championId')
        orders.append(team+'_'+lane+'_spell1Id')
        orders.append(team+'_'+lane+'_spell2Id')
        
match_train_df = match_train_df[orders]

match_train_X_df = pd.get_dummies(match_train_df, columns=match_train_df.columns)
match_train_y_df = match_df['win']

match_train_df.head()

,team1_TOP_championId,team1_TOP_spell1Id,team1_TOP_spell2Id,team1_JUG_championId,team1_JUG_spell1Id,team1_JUG_spell2Id,team1_MID_championId,team1_MID_spell1Id,team1_MID_spell2Id,team1_ADC_championId,...,team2_JUG_spell2Id,team2_MID_championId,team2_MID_spell1Id,team2_MID_spell2Id,team2_ADC_championId,team2_ADC_spell1Id,team2_ADC_spell2Id,team2_SUP_championId,team2_SUP_spell1Id,team2_SUP_spell2Id
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,68,12,4,106,4,11,61,1,4,22,...,4,142,12,4,235,4,7,875,4,14


In [131]:
match_train_X_df.head()

,team1_TOP_championId_68,team1_TOP_spell1Id_12,team1_TOP_spell2Id_4,team1_JUG_championId_106,team1_JUG_spell1Id_4,team1_JUG_spell2Id_11,team1_MID_championId_61,team1_MID_spell1Id_1,team1_MID_spell2Id_4,team1_ADC_championId_22,...,team2_JUG_spell2Id_4,team2_MID_championId_142,team2_MID_spell1Id_12,team2_MID_spell2Id_4,team2_ADC_championId_235,team2_ADC_spell1Id_4,team2_ADC_spell2Id_7,team2_SUP_championId_875,team2_SUP_spell1Id_4,team2_SUP_spell2Id_14
gameId,,,,,,,,,,,,,,,,,,,,,
4530461969,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [128]:
match_train_y_df.head()

gameId
4530461969    team1
Name: win, dtype: object

In [ ]:
match_df['team2_TOP_summonerName']

In [89]:
match_df['participantIdentities'].values[0][0]

{'participantId': 1,
 'player': {'platformId': 'KR',
  'accountId': 'B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA',
  'summonerName': '사 용 수',
  'summonerId': 'XoeObrc8p2RKmU1fglPJXxMrJyhQjs2kupP9_jiMlcZ1X4E',
  'currentPlatformId': 'KR',
  'currentAccountId': 'B0YBK1wCNNubEk-tRp7qmN601ubjdWLO6C7bkRFH50clSkA',
  'matchHistoryUri': '/v1/stats/player_history/KR/204196076',
  'profileIcon': 3872}}